
## Problem statement
We have to build collaborative filtering models for recommending product items. The steps below aim to recommend users their top 10 items to place into their basket. The final output will be a csv file in the `output` folder, and a function that searches for a recommendation list based on a speficied user:
* Input: user - customer ID
* Returns: ranked list of items (product IDs), that the user is most likely to want to put in his/her (empty) "basket"

## 1. Import modules
* `pandas` and `numpy` for data manipulation
* `turicreate` for performing model selection and evaluation
* `sklearn` for splitting the data into train and test set

In [39]:
! pip install turicreate

In [0]:

import pandas as pd
import numpy as np
import time
import turicreate as tc
from sklearn.model_selection  import train_test_split


## 2. Load data
Two datasets are used in this exercise: 
* `recommend_1.csv` consisting of a list of 1000 customer IDs to recommend as output
* `trx_data.csv` consisting of user transactions

The format is as follows.

In [0]:
customers = pd.read_csv('recommend_1.csv', error_bad_lines=False)
transactions = pd.read_csv('trx_data.csv', error_bad_lines=False)

In [42]:
print(customers.shape)
customers.head()

(1000, 1)


,customerId
0,1553
1,20400
2,19750
3,6334
4,27773


In [43]:
print(transactions.shape)
transactions.head()

(62483, 2)


,customerId,products
0,0,20
1,1,2|2|23|68|68|111|29|86|107|152
2,2,111|107|29|11|11|11|33|23
3,3,164|227
4,5,2|2


## 3. Data preparation
* Our goal here is to break down each list of items in the `products` column into rows and count the number of products bought by a user

In [44]:
# example 1: split product items
transactions['products'] = transactions['products'].apply(lambda x: [int(i) for i in x.split('|')])
transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index()

,customerId,0,1,2,3,4,5,6,7,8,9
0,0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2.0,2.0,23.0,68.0,68.0,111.0,29.0,86.0,107.0,152.0


In [45]:
# example 2: organize a given table into a dataframe with customerId, single productId, and purchase count
pd.melt(transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})

,customerId,productId,purchase_count
0,0,20.0,1
1,1,2.0,2
2,1,23.0,1
3,1,29.0,1
4,1,68.0,2
5,1,86.0,1
6,1,107.0,1
7,1,111.0,1
8,1,152.0,1


### 3.1. Create data with user, item, and target field
* This table will be an input for our modeling later
    * In this case, our user is `customerId`, `productId`, and `purchase_count`

In [46]:
s=time.time()

data = pd.melt(transactions.set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})
data['productId'] = data['productId'].astype(np.int64)

print("Execution time:", round((time.time()-s)/60,2), "minutes")

Execution time: 0.3 minutes


In [47]:
print(data.shape)
data.head()

(133585, 3)


,customerId,productId,purchase_count
0,0,1,2
1,0,13,1
2,0,19,3
3,0,20,1
4,0,31,2


### 3.2. Create dummy
* Dummy for marking whether a customer bought that item or not.
* If one buys an item, then `purchase_dummy` are marked as 1
* Why create a dummy instead of normalizing it, you ask?
    * Normalizing the purchase count, say by each user, would not work because customers may have different buying frequency don't have the same taste
    * However, we can normalize items by purchase frequency across all users, which is done in section 3.3. below.

In [0]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy

In [0]:
data_dummy = create_data_dummy(data)

### 3.3. Normalize item values across users
* To do this, we normalize purchase frequency of each item across users by first creating a user-item matrix as follows

In [50]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
df_matrix.head()

productId,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
customerId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
(df_matrix.shape)

(24429, 300)

In [52]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
print(df_matrix_norm.shape)
df_matrix_norm.head()

(24429, 300)


productId,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
customerId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,NaN,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.142857,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.571429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,0.166667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.022222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
# create a table for input to the modeling

d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm.head()

(133585, 3)


,customerId,productId,scaled_purchase_freq
9,9,0,0.133333
25,25,0,0.133333
32,33,0,0.133333
35,36,0,0.133333
43,44,0,0.133333


#### Define a function for normalizing data

In [0]:
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()

* We can normalize the their purchase history, from 0-1 (with 1 being the most number of purchase for an item and 0 being 0 purchase count for that item).

## 4. Split train and test set
* Splitting the data into training and testing sets is an important part of evaluating predictive modeling, in this case a collaborative filtering model. Typically, we use a larger portion of the data for training and a smaller portion for testing. 
* We use 80:20 ratio for our train-test set size.
* Our training portion will be used to develop a predictive model, while the other to evaluate the model's performance.
* Now that we have three datasets with purchase counts, purchase dummy, and scaled purchase counts, we would like to split each.

In [55]:
train, test = train_test_split(data, test_size = .2)
print(train.shape, test.shape)

(106868, 3) (26717, 3)


In [0]:
# Using turicreate library, we convert dataframe to SFrame - this will be useful in the modeling part

train_data = tc.SFrame(train)
test_data = tc.SFrame(test)

In [57]:
train_data

customerId,productId,purchase_count
2946,81,2
1297,275,1
704,143,1
8743,139,3
25717,66,1
1441,1,1
5202,153,1
3429,273,4
24955,171,3
16922,108,2


In [58]:
test_data

customerId,productId,purchase_count
2317,62,4
14112,218,1
1441,8,1
7223,85,1
1084,15,1
19014,1,1
4404,2,5
1507,6,8
25364,135,1
26358,268,1


#### Define a `split_data` function for splitting data to training and test set

In [0]:
# We can define a function for this step as follows

def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [0]:
# lets try with both dummy table and scaled/normalized purchase table

train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

## 5. Baseline Model
Before running a more complicated approach such as collaborative filtering, we would like to use a baseline model to compare and evaluate models. Since baseline typically uses a very simple approach, techniques used beyond this approach should be chosen if they show relatively better accuracy and complexity.

### 5.1. Using a Popularity model as a baseline
* The popularity model takes the most popular items for recommendation. These items are products with the highest number of sells across customers.
* We use `turicreate` library for running and evaluating both baseline and collaborative filtering models below
* Training data is used for model selection

#### Using purchase counts

In [0]:
# variables to define field names
user_id = 'customerId'
item_id = 'productId'
target = 'purchase_count'
users_to_recommend = list(transactions[user_id])
n_rec = 10 # number of items to recommend
n_display = 30

In [62]:
popularity_model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Preparing data set.

Data has 106868 observations with 23366 users and 300 items.

Data prepared in: 0.121398s

106868 observations to process; with 300 unique items.

In [63]:
# Get recommendations for a list of users to recommend (from customers file)
# Printed below is head / top 30 rows for first 3 customers with 10 recommendations each

popularity_recomm = popularity_model.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm.print_rows(n_display)

recommendations finished on 1000/62483 queries. users per second: 89405.5

recommendations finished on 2000/62483 queries. users per second: 101807

recommendations finished on 3000/62483 queries. users per second: 111682

recommendations finished on 4000/62483 queries. users per second: 118259

recommendations finished on 5000/62483 queries. users per second: 118883

...

recommendations finished on 59000/62483 queries. users per second: 117663

recommendations finished on 60000/62483 queries. users per second: 117672

recommendations finished on 61000/62483 queries. users per second: 117497

recommendations finished on 62000/62483 queries. users per second: 117311

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|     0      |    132    | 3.1481481481481484 |  1   |
|     0      |     34    | 3.0689655172413794 |  2   |
|     0      |     37    | 3.0077220077220077 |  3   |
|     0      |     0     | 2.9549376797698947 |  4   |
|     0      |    248    | 2.877551020408163  |  5   |
|     0      |     3     | 2.831983805668016  |  6   |
|     0      |    110    | 2.7966101694915255 |  7   |
|     0      |     27    | 2.6946564885496183 |  8   |
|     0      |     10    | 2.641833810888252  |  9   |
|     0      |    230    |  2.56390977443609  |  10  |
|     1      |    132    | 3.1481481481481484 |  1   |
|     1      |     34    | 3.0689655172413794 |  2   |
|     1      |     37    | 3.0077220077220077 |  3   |
|     1      |     0     | 2.9549376797698947 |  4   |
|     1      |    248    | 2.877551020408163  |  5   |
|     1   

#### Define a `model` function for model selection

In [0]:
# Since turicreate is very accessible library, we can define a model selection function as below

def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [0]:
# variables to define field names
# constant variables include:
user_id = 'customerId'
item_id = 'productId'
users_to_recommend = list(customers[user_id])
n_rec = 10 # number of items to recommend
n_display = 30 # to print the head / first few rows in a defined dataset

#### Using purchase dummy

In [66]:
# these variables will change accordingly
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23332 users and 300 items.

Data prepared in: 0.124998s

106868 observations to process; with 300 unique items.

recommendations finished on 1000/1000 queries. users per second: 104679

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|    1553    |    170    |  1.0  |  1   |
|    1553    |     96    |  1.0  |  2   |
|    1553    |     5     |  1.0  |  3   |
|    1553    |    107    |  1.0  |  4   |
|    1553    |     23    |  1.0  |  5   |
|    1553    |    129    |  1.0  |  6   |
|    1553    |    189    |  1.0  |  7   |
|    1553    |    144    |  1.0  |  8   |
|    1553    |     65    |  1.0  |  9   |
|    1553    |    106    |  1.0  |  10  |
|   20400    |     96    |  1.0  |  1   |
|   20400    |     5     |  1.0  |  2   |
|   20400    |    107    |  1.0  |  3   |
|   20400    |     14    |  1.0  |  4   |
|   20400    |     23    |  1.0  |  5   |
|   20400    |    129    |  1.0  |  6   |
|   20400    |    189    |  1.0  |  7   |
|   20400    |    144    |  1.0  |  8   |
|   20400    |     65    |  1.0  |  9   |
|   20400    |    106    |  1.0  |  10  |
|   19750    |     96    |  1.0  |

#### Using normalized purchase count

In [67]:
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23292 users and 300 items.

Data prepared in: 0.126998s

106868 observations to process; with 300 unique items.

recommendations finished on 1000/1000 queries. users per second: 88699.7

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|    1553    |    226    |  0.7789115646258503 |  1   |
|    1553    |    230    | 0.33383458646616493 |  2   |
|    1553    |    247    | 0.33333333333333326 |  3   |
|    1553    |    125    | 0.26857142857142824 |  4   |
|    1553    |    248    | 0.26063829787234044 |  5   |
|    1553    |    294    | 0.25238095238095204 |  6   |
|    1553    |    155    |  0.2482758620689654 |  7   |
|    1553    |    204    | 0.23655913978494605 |  8   |
|    1553    |    276    | 0.23214285714285715 |  9   |
|    1553    |    165    |        0.225        |  10  |
|   20400    |    226    |  0.7789115646258503 |  1   |
|   20400    |    230    | 0.33383458646616493 |  2   |
|   20400    |    247    | 0.33333333333333326 |  3   |
|   20400    |    125    | 0.26857142857142824 |  4   |
|   20400    |    248    | 0.26063829787234044 |

#### Notes
* Once we created the model, we predicted the recommendation items using scores by popularity. As you can tell for each model results above, the rows show the first 30 records from 1000 users with 10 recommendations. These 30 records include 3 users and their recommended items, along with score and descending ranks. 
* In the result, although different models have different recommendation list, each user is recommended the same list of 10 items. This is because popularity is calculated by taking the most popular items across all users.
* If a grouping example below, products 132, 248, 37, and 34 are the most popular (best-selling) across customers. Using their purchase counts divided by the number of customers, we see that these products are at least bought 3 times on average in the training set of transactions (same as the first popularity measure on `purchase_count` variable)

In [68]:
train.groupby(by=item_id)['purchase_count'].mean().sort_values(ascending=False).head(20)

productId
132    3.148148
34     3.068966
37     3.007722
0      2.954938
248    2.877551
3      2.831984
110    2.796610
27     2.694656
10     2.641834
230    2.563910
32     2.551546
226    2.546763
245    2.527473
58     2.500000
252    2.471698
68     2.418803
18     2.405612
54     2.404412
87     2.401674
91     2.378486
Name: purchase_count, dtype: float64

## 6. Collaborative Filtering Model

* In collaborative filtering, we would recommend items based on how similar users purchase items. For instance, if customer 1 and customer 2 bought similar items, e.g. 1 bought X, Y, Z and 2 bought X, Y, we would recommend an item Z to customer 2.

* To define similarity across users, we use the following steps:
    1. Create a user-item matrix, where index values represent unique customer IDs and column values represent unique product IDs
    
    2. Create an item-to-item similarity matrix. The idea is to calculate how similar a product is to another product. There are a number of ways of calculating this. In steps 6.1 and 6.2, we use cosine and pearson similarity measure, respectively.  
    
        * To calculate similarity between products X and Y, look at all customers who have rated both these items. For example, both X and Y have been rated by customers 1 and 2. 
        * We then create two item-vectors, v1 for item X and v2 for item Y, in the user-space of (1, 2) and then find the `cosine` or `pearson` angle/distance between these vectors. A zero angle or overlapping vectors with cosine value of 1 means total similarity (or per user, across all items, there is same rating) and an angle of 90 degree would mean cosine of 0 or no similarity.
        
    3. For each customer, we then predict his likelihood to buy a product (or his purchase counts) for products that he had not bought. 
    
        * For our example, we will calculate rating for user 2 in the case of item Z (target item). To calculate this we weigh the just-calculated similarity-measure between the target item and other items that customer has already bought. The weighing factor is the purchase counts given by the user to items already bought by him. 
        * We then scale this weighted sum with the sum of similarity-measures so that the calculated rating remains within a predefined limits. Thus, the predicted rating for item Z for user 2 would be calculated using similarity measures.

* While I wrote python scripts for all the process including finding similarity using python scripts (which can be found in `scripts` folder, we can use `turicreate` library for now to capture different measures like using `cosine` and `pearson` distance, and evaluate the best model.

### 6.1. `Cosine` similarity
* Similarity is the cosine of the angle between the 2 vectors of the item vectors of A and B
* It is defined by the following formula
![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTnRHSAx1c084UXF2wIHYwaHJLmq2qKtNk_YIv3RjHUO00xwlkt)
* Closer the vectors, smaller will be the angle and larger the cosine

#### Using purchase count

In [69]:
# these variables will change accordingly
name = 'cosine'
target = 'purchase_count'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23366 users and 300 items.

Data prepared in: 0.125327s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.08326s

recommendations finished on 1000/1000 queries. users per second: 28233.4

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|    1553    |     2     | 0.13541358709335327  |  1   |
|    1553    |     35    | 0.11725354194641113  |  2   |
|    1553    |     17    | 0.08008340994517009  |  3   |
|    1553    |     1     | 0.07827071348826091  |  4   |
|    1553    |     5     |  0.0740700364112854  |  5   |
|    1553    |     21    | 0.06394805510838826  |  6   |
|    1553    |     47    | 0.06064931551615397  |  7   |
|    1553    |     13    | 0.057833572228749595 |  8   |
|    1553    |    150    | 0.05629505713780721  |  9   |
|    1553    |     15    | 0.05536544322967529  |  10  |
|   20400    |    280    |  0.0845455527305603  |  1   |
|   20400    |    284    | 0.039641380310058594 |  2   |
|   20400    |     1     | 0.038047730922698975 |  3   |
|   20400    |     26    | 0.03721386194229126  |  4   |
|   20400    |    246    |  0.0

#### Using purchase dummy

In [70]:
# these variables will change accordingly
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23332 users and 300 items.

Data prepared in: 0.125691s

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.072591s

recommendations finished on 1000/1000 queries. users per second: 33973.2

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|    1553    |     2     | 0.11079622805118561  |  1   |
|    1553    |     35    | 0.10909666121006012  |  2   |
|    1553    |     1     |  0.0906706154346466  |  3   |
|    1553    |     5     | 0.08284689486026764  |  4   |
|    1553    |     8     | 0.06188082695007324  |  5   |
|    1553    |     33    | 0.059453919529914856 |  6   |
|    1553    |     36    | 0.05837637186050415  |  7   |
|    1553    |    106    | 0.05349157750606537  |  8   |
|    1553    |     21    | 0.05245846509933472  |  9   |
|    1553    |     15    | 0.050799086689949036 |  10  |
|   20400    |     26    | 0.049696147441864014 |  1   |
|   20400    |     58    | 0.04531306028366089  |  2   |
|   20400    |    273    | 0.040882885456085205 |  3   |
|   20400    |    113    | 0.04074352979660034  |  4   |
|   20400    |    215    | 0.03

#### Using normalized purchase count

In [71]:
name = 'cosine'
target = 'scaled_purchase_freq'
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23292 users and 300 items.

Data prepared in: 0.134326s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.076313s

recommendations finished on 1000/1000 queries. users per second: 42414.2

+------------+-----------+-----------------------+------+
| customerId | productId |         score         | rank |
+------------+-----------+-----------------------+------+
|    1553    |     7     |          0.0          |  1   |
|    1553    |     11    |          0.0          |  2   |
|    1553    |    176    |          0.0          |  3   |
|    1553    |    244    |          0.0          |  4   |
|    1553    |     39    |          0.0          |  5   |
|    1553    |     43    |          0.0          |  6   |
|    1553    |     12    |          0.0          |  7   |
|    1553    |    177    |          0.0          |  8   |
|    1553    |    273    |          0.0          |  9   |
|    1553    |    114    |          0.0          |  10  |
|   20400    |     1     |  0.003978612422943115 |  1   |
|   20400    |     2     |  0.003670961856842041 |  2   |
|   20400    |     13    |  0.002444273233413696 |  3   |
|   20400    |     0     |  0.002372108697891235 |  4   |
|   20400    |

### 6.2. `Pearson` similarity
* Similarity is the pearson coefficient between the two vectors.
* It is defined by the following formula
![](http://critical-numbers.group.shef.ac.uk/glossary/images/correlationKT1.png)

#### Using purchase count

In [72]:
# these variables will change accordingly
name = 'pearson'
target = 'purchase_count'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23366 users and 300 items.

Data prepared in: 0.139597s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.1053s

recommendations finished on 1000/1000 queries. users per second: 59995.2

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|    1553    |    132    | 3.1481481481481484 |  1   |
|    1553    |     34    | 3.068965517241379  |  2   |
|    1553    |     37    | 3.0048790449801563 |  3   |
|    1553    |     0     | 2.9529009890579156 |  4   |
|    1553    |    248    | 2.8775510204081636 |  5   |
|    1553    |     3     | 2.8319838056680178 |  6   |
|    1553    |    110    |  2.79129899827774  |  7   |
|    1553    |     27    | 2.694656488549618  |  8   |
|    1553    |     10    | 2.6418338108882513 |  9   |
|    1553    |    230    | 2.5573732509350124 |  10  |
|   20400    |    132    | 3.1481481481481484 |  1   |
|   20400    |     34    | 3.068965517241379  |  2   |
|   20400    |     37    | 3.007722007722008  |  3   |
|   20400    |     0     | 2.9536013436340265 |  4   |
|   20400    |    248    | 2.8756892097239595 |  5   |
|   20400 

#### Using purchase dummy

In [73]:
# these variables will change accordingly
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23332 users and 300 items.

Data prepared in: 0.13989s

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.091272s

recommendations finished on 1000/1000 queries. users per second: 57280.3

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|    1553    |    170    |  0.0  |  1   |
|    1553    |     96    |  0.0  |  2   |
|    1553    |     5     |  0.0  |  3   |
|    1553    |    107    |  0.0  |  4   |
|    1553    |     23    |  0.0  |  5   |
|    1553    |    129    |  0.0  |  6   |
|    1553    |    189    |  0.0  |  7   |
|    1553    |    144    |  0.0  |  8   |
|    1553    |     65    |  0.0  |  9   |
|    1553    |    106    |  0.0  |  10  |
|   20400    |     96    |  0.0  |  1   |
|   20400    |     5     |  0.0  |  2   |
|   20400    |    107    |  0.0  |  3   |
|   20400    |     14    |  0.0  |  4   |
|   20400    |     23    |  0.0  |  5   |
|   20400    |    129    |  0.0  |  6   |
|   20400    |    189    |  0.0  |  7   |
|   20400    |    144    |  0.0  |  8   |
|   20400    |     65    |  0.0  |  9   |
|   20400    |    106    |  0.0  |  10  |
|   19750    |     96    |  0.0  |

#### Using normalized purchase count

In [74]:
name = 'pearson'
target = 'scaled_purchase_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23292 users and 300 items.

Data prepared in: 0.128545s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.104816s

recommendations finished on 1000/1000 queries. users per second: 53995.7

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|    1553    |    226    |  0.7789115646258504 |  1   |
|    1553    |    230    |  0.3336027243979892 |  2   |
|    1553    |    247    |  0.3333333333333333 |  3   |
|    1553    |    125    |  0.2682890575272695 |  4   |
|    1553    |    248    | 0.26016875632265773 |  5   |
|    1553    |    294    |  0.2523809523809523 |  6   |
|    1553    |    155    | 0.24827586206896557 |  7   |
|    1553    |    204    | 0.23655913978494625 |  8   |
|    1553    |    276    | 0.23183422429221007 |  9   |
|    1553    |    165    |  0.224996268749237  |  10  |
|   20400    |    226    |  0.7788998439685018 |  1   |
|   20400    |    230    | 0.33382817181429475 |  2   |
|   20400    |    247    |  0.3333313675721486 |  3   |
|   20400    |    125    |  0.268570247207369  |  4   |
|   20400    |    248    | 0.26063274987200463 |

#### Note
* In collaborative filtering above, we used two approaches: cosine and pearson distance. We also got to apply them to three training datasets with normal counts, dummy, or normalized counts of items purchase.
* We can see that the recommendations are different for each user. This suggests that personalization does exist. 
* But how good is this model compared to the baseline, and to each other? We need some means of evaluating a recommendation engine. Lets focus on that in the next section.

## 7. Model Evaluation
For evaluating recommendation engines, we can use the concept of precision-recall.

* RMSE (Root Mean Squared Errors)
    * Measures the error of predicted values
    * Lesser the RMSE value, better the recommendations
* Recall
    * What percentage of products that a user buys are actually recommended?
    * If a customer buys 5 products and the recommendation decided to show 3 of them, then the recall is 0.6
* Precision
    * Out of all the recommended items, how many the user actually liked?
    * If 5 products were recommended to the customer out of which he buys 4 of them, then precision is 0.8
    
* Why are both recall and precision important?
    * Consider a case where we recommend all products, so our customers will surely cover the items that they liked and bought. In this case, we have 100% recall! Does this mean our model is good?
    * We have to consider precision. If we recommend 300 items but user likes and buys only 3 of them, then precision is 0.1%! This very low precision indicates that the model is not great, despite their excellent recall.
    * So our aim has to be optimizing both recall and precision (to be close to 1 as possible).

Lets compare all the models we have built based on precision-recall characteristics:

In [0]:
# create initial callable variables

models_w_counts = [popularity_model, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]

names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']

#### Models on purchase counts

In [76]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)

PROGRESS: Evaluate model Popularity Model on Purchase Counts


recommendations finished on 1000/13875 queries. users per second: 54662.7

recommendations finished on 2000/13875 queries. users per second: 53433.1

recommendations finished on 3000/13875 queries. users per second: 54961.2

recommendations finished on 4000/13875 queries. users per second: 56873.1

...

recommendations finished on 10000/13875 queries. users per second: 41735.7

recommendations finished on 11000/13875 queries. users per second: 41728

recommendations finished on 12000/13875 queries. users per second: 41968.8

recommendations finished on 13000/13875 queries. users per second: 41077.9


Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    | 0.0012972972972972983 | 0.0006942942942942942 |
|   2    | 0.0033153153153153204 |  0.003421138021138024 |
|   3    | 0.0036036036036036154 |  0.005564252564252572 |
|   4    |  0.007369369369369374 |  0.01577957540898721  |
|   5    |  0.006082882882882929 |   0.0160830789124907  |
|   6    |  0.006198198198198208 |  0.019871939601351356 |
|   7    |  0.005734877734877742 |  0.021488642018053776 |
|   8    | 0.0053693693693693725 |    0.02291941824883   |
|   9    | 0.0053253253253253835 |  0.026375960505372213 |
|   10   |  0.005138738738738745 |  0.02842309335250509  |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 1.129987675410589

Per User RMSE (best)
+------------+-----------------------+-------

#### Models on purchase dummy

In [77]:
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)

PROGRESS: Evaluate model Popularity Model on Purchase Dummy


recommendations finished on 1000/13926 queries. users per second: 53347.6

recommendations finished on 2000/13926 queries. users per second: 56309.5

recommendations finished on 3000/13926 queries. users per second: 60494.8

recommendations finished on 4000/13926 queries. users per second: 63016.9

...

recommendations finished on 10000/13926 queries. users per second: 55675.6

recommendations finished on 11000/13926 queries. users per second: 56220.5

recommendations finished on 12000/13926 queries. users per second: 56776

recommendations finished on 13000/13926 queries. users per second: 54902.6


Precision and recall summary statistics by cutoff
+--------+-----------------------+----------------------+
| cutoff |     mean_precision    |     mean_recall      |
+--------+-----------------------+----------------------+
|   1    |  0.006319115323854659 | 0.003768439301614659 |
|   2    | 0.0054574177796926715 | 0.006137209946451654 |
|   3    |  0.005864330508880261 | 0.009611697544161999 |
|   4    |  0.006480683613385036 | 0.013911632294333716 |
|   5    |  0.006290392072382634 | 0.016672778321672483 |
|   6    |  0.006139595002154268 | 0.018796446736810135 |
|   7    |  0.006196015674688649 | 0.02211241763141087  |
|   8    |  0.006157547034324286 | 0.02520361768512707  |
|   9    |  0.006151563023601014 | 0.028550047453357784 |
|   10   |  0.006161137440758263 | 0.032378648599925355 |
+--------+-----------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 1.0

Per User RMSE (best)
+------------+------+-------+
| customerId | rmse | count |
+------------

#### Models on normalized purchase frequency

In [78]:
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

PROGRESS: Evaluate model Popularity Model on Scaled Purchase Counts


recommendations finished on 1000/13922 queries. users per second: 51840.3

recommendations finished on 2000/13922 queries. users per second: 49870.3

recommendations finished on 3000/13922 queries. users per second: 52114.1

recommendations finished on 4000/13922 queries. users per second: 53438.1

...

recommendations finished on 10000/13922 queries. users per second: 40555

recommendations finished on 11000/13922 queries. users per second: 40961.6

recommendations finished on 12000/13922 queries. users per second: 40800.4

recommendations finished on 13000/13922 queries. users per second: 41138.5


Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    |  0.002370349087774743 |  0.001299478666236315 |
|   2    |  0.002549920988363739 | 0.0031211358355446826 |
|   3    | 0.0023703490877747388 |  0.004293141773388864 |
|   4    | 0.0022805631374802505 |  0.005541366006816372 |
|   5    | 0.0019393765263611487 |  0.00584663823781767  |
|   6    |  0.002071062586793091 |  0.007344298983431291 |
|   7    | 0.0018778090175877817 |  0.007787243004884153 |
|   8    |  0.001831633386007758 |  0.008716826876933181 |
|   9    | 0.0018835097128445718 |  0.009875863733068159 |
|   10   | 0.0021764114351386418 |  0.012181598031332274 |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.13464882193962685

Per User RMSE (best)
+------------+------+-------+
| customerId 

## 8. Model Selection
### 8.1. Evaluation summary
* Based on RMSE


    1. Popularity on purchase counts: 1.1111750034210488
    2. Cosine similarity on purchase counts: 1.9230643981653215
    3. Pearson similarity on purchase counts: 1.9231102838192284
    
    4. Popularity on purchase dummy: 0.9697374361161925
    5. Cosine similarity on purchase dummy: 0.9697509978436404
    6. Pearson similarity on purchase dummy: 0.9697745320187097
    
    7. Popularity on scaled purchase counts: 0.16230660626840343
    8. Cosine similarity on scaled purchase counts: 0.16229800354111104
    9. Pearson similarity on scaled purchase counts: 0.1622982668334026
    
* Based on Precision and Recall
![](../images/model_comparisons.png)


#### Notes

* Popularity v. Collaborative Filtering: We can see that the collaborative filtering algorithms work better than popularity model for purchase counts. Indeed, popularity model doesn’t give any personalizations as it only gives the same list of recommended items to every user.
* Precision and recall: Looking at the summary above, we see that the precision and recall for Purchase Counts > Purchase Dummy > Normalized Purchase Counts. However, because the recommendation scores for the normalized purchase data is zero and constant, we choose the dummy. In fact, the RMSE isn’t much different between models on the dummy and those on the normalized data.
* RMSE: Since RMSE is higher using pearson distance thancosine, we would choose model the smaller mean squared errors, which in this case would be cosine.
Therefore, we select the Cosine similarity on Purchase Dummy approach as our final model.

## 8. Final Output
* In this step, we would like to manipulate format for recommendation output to one we can export to csv, and also a function that will return recommendation list given a customer ID.
* We need to first rerun the model using the whole dataset, as we came to a final model using train data and evaluated with test set.

In [79]:
users_to_recommend = list(customers[user_id])

final_model = tc.item_similarity_recommender.create(tc.SFrame(data_dummy), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='purchase_dummy', 
                                            similarity_type='cosine')

recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 133585 observations with 24429 users and 300 items.

Data prepared in: 0.136733s

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.101247s

recommendations finished on 1000/1000 queries. users per second: 60350

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|    1553    |     2     | 0.10592188835144042  |  1   |
|    1553    |     1     | 0.10348175764083863  |  2   |
|    1553    |     35    |  0.0845762014389038  |  3   |
|    1553    |     33    |  0.0668614387512207  |  4   |
|    1553    |     5     | 0.06496070623397827  |  5   |
|    1553    |     61    | 0.060317397117614746 |  6   |
|    1553    |     15    | 0.05949603319168091  |  7   |
|    1553    |     17    |  0.0519999623298645  |  8   |
|    1553    |     11    | 0.050322222709655764 |  9   |
|    1553    |     19    | 0.049363481998443606 |  10  |
|   20400    |     26    | 0.05812269449234009  |  1   |
|   20400    |     6     | 0.05361741781234741  |  2   |
|   20400    |    113    | 0.05312788486480713  |  3   |
|   20400    |     1     | 0.05210459232330322  |  4   |
|   20400    |     15    | 0.04

### 8.1. CSV output file

In [80]:
df_rec = recom.to_dataframe()
print(df_rec.shape)
df_rec.head()

(10000, 4)


,customerId,productId,score,rank
0,1553,2,0.105922,1
1,1553,1,0.103482,2
2,1553,35,0.084576,3
3,1553,33,0.066861,4
4,1553,5,0.064961,5


In [0]:
df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id].transform(lambda x: '|'.join(x.astype(str)))
df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates().sort_values('customerId').set_index('customerId')

#### Define a function to create a desired output

In [0]:
def create_output(model, users_to_recommend, n_rec, print_csv=True):
    recomendation = model.recommend(users=users_to_recommend, k=n_rec)
    df_rec = recomendation.to_dataframe()
    df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))
    df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates() \
        .sort_values('customerId').set_index('customerId')
    if print_csv:
        df_output.to_csv('../option1_recommendation.csv')
        print("An output file can be found in 'output' folder with name 'option1_recommendation.csv'")
    return df_output

In [85]:
df_output = create_output(pear_norm, users_to_recommend, n_rec, print_csv=True)
print(df_output.shape)
df_output.head()

recommendations finished on 1000/1000 queries. users per second: 43308.8

An output file can be found in 'output' folder with name 'option1_recommendation.csv'
(1000, 1)


,recommendedProducts
customerId,
4,226|230|247|125|248|294|155|204|276|165
11,226|230|247|125|248|294|155|204|276|165
12,226|230|247|125|248|294|155|204|276|165
16,226|230|247|125|248|294|155|204|276|72
21,226|230|247|125|248|294|155|204|276|72


### 8.2. Customer recommendation function

In [0]:
def customer_recomendation(customer_id):
    if customer_id not in df_output.index:
        print('Customer not found.')
        return customer_id
    return df_output.loc[customer_id]

In [0]:
customer_recomendation(4)

In [0]:
customer_recomendation(21)

## Summary
In this exercise, we were able to traverse a step-by-step process for making recommendations to customers. We used Collaborative Filtering approaches with `cosine` and `pearson` measure and compare the models with our baseline popularity model. We also prepared three sets of data that include regular buying count, buying dummy, as well as normalized purchase frequency as our target variable. Using RMSE, precision and recall, we evaluated our models and observed the impact of personalization. Finally, we selected the Cosine approach in dummy purchase data. 